# Retrieve OpenRoads from OS API

Dowload OpenRoads from OS API, save zip, open it in GeoPandas and save. Now to parquet, then to PostGIS.

In [42]:
import requests
import geopandas as gpd

In [43]:
response = requests.get('https://api.os.uk/downloads/v1/products')
json = response.json()
json

[{'id': '250kScaleColourRaster',
  'name': '1:250 000 Scale Colour Raster™',
  'description': 'Get the regional view of towns and villages, roads and places of interest.',
  'version': '2020-06',
  'url': 'https://api.os.uk/downloads/v1/products/250kScaleColourRaster'},
 {'id': 'BoundaryLine',
  'name': 'Boundary-Line™',
  'description': 'From Euro constituencies to council wards, Boundary-Line™ maps every administrative boundary in detail for you.',
  'version': '2020-05',
  'url': 'https://api.os.uk/downloads/v1/products/BoundaryLine'},
 {'id': 'CodePointOpen',
  'name': 'Code-Point® Open',
  'description': 'Get started with geographical analysis, simple route planning and asset management.',
  'version': '2020-05',
  'url': 'https://api.os.uk/downloads/v1/products/CodePointOpen'},
 {'id': 'GBOverviewMaps',
  'name': 'GB Overview Maps',
  'description': 'Our simplest maps of the British Isles.',
  'version': '2014-11',
  'url': 'https://api.os.uk/downloads/v1/products/GBOverviewMaps'

In [47]:
for product in json:
    if product['id'] == 'OpenRoads':
        url = product['url']


In [48]:
roads = requests.get(url)
roads_json = roads.json()
roads_json

{'id': 'OpenRoads',
 'name': 'OS Open Roads',
 'description': 'Get a high-level view of the road network, from motorways to country lanes.',
 'version': '2020-04',
 'documentationUrl': 'https://www.ordnancesurvey.co.uk/business-government/products/open-map-roads',
 'dataStructures': ['Vector'],
 'category': 'Networks',
 'formats': [{'format': 'ESRI® Shapefile'},
  {'format': 'GML', 'subformat': '3'},
  {'format': 'GeoPackage'}],
 'url': 'https://api.os.uk/downloads/v1/products/OpenRoads',
 'imageCount': 3,
 'imageTemplate': 'https://api.os.uk/downloads/v1/products/OpenRoads/images/{index}',
 'downloadsUrl': 'https://api.os.uk/downloads/v1/products/OpenRoads/downloads',
 'areas': ['GB']}

In [49]:
download = requests.get(roads_json['downloadsUrl'])
download_json = download.json()
download_json

[{'md5': '60286049CB76C9D6364F0FE87090202C',
  'size': 541236838,
  'url': 'https://api.os.uk/downloads/v1/products/OpenRoads/downloads?area=GB&format=ESRI%C2%AE+Shapefile&redirect',
  'format': 'ESRI® Shapefile',
  'area': 'GB',
  'fileName': 'oproad_essh_gb.zip'},
 {'md5': '84E66CAFCEA9DD695A68540DE9379D4C',
  'size': 542188036,
  'url': 'https://api.os.uk/downloads/v1/products/OpenRoads/downloads?area=GB&format=GML&subformat=3&redirect',
  'format': 'GML',
  'subformat': '3',
  'area': 'GB',
  'fileName': 'oproad_gml3_gb.zip'},
 {'md5': 'E1FC93742CAAE7EF70C675749077AB42',
  'size': 931331115,
  'url': 'https://api.os.uk/downloads/v1/products/OpenRoads/downloads?area=GB&format=GeoPackage&redirect',
  'format': 'GeoPackage',
  'area': 'GB',
  'fileName': 'oproad_gpkg_gb.zip'}]

In [50]:
for fileformat in download_json:
    if fileformat['format'] == 'GeoPackage':
        with open(fileformat['fileName'], "wb") as down:
            down.write(requests.get(fileformat['url']).content)
            down.close()

In [51]:
import zipfile

In [52]:
gpkg = zipfile.ZipFile(fileformat['fileName'])

In [53]:
gpkg.namelist()

['readme.txt', 'data/oproad_gb.gpkg', 'doc/licence.txt']

In [54]:
gdf = gpd.read_file('zip://' + fileformat['fileName'] + '!' + gpkg.namelist()[1])

In [55]:
gdf.head()

,id,endNode,startNode,roadNumberTOID,roadNameTOID,fictitious,roadClassification,roadFunction,formOfWay,length,...,loop,primaryRoute,trunkRoad,roadClassificationNumber,name1,name1_lang,name2,name2_lang,roadStructure,geometry
0,id04AE862E-2A92-41BC-BBE0-6AF05DDDE57E,idD6A4C11F-3E69-4E22-9711-2DE393FCF0A2,idF4241DAB-6DC5-4C0E-90FA-161BF2270663,None,None,False,Unknown,Minor Road,Single Carriageway,500,...,False,False,False,None,None,None,None,None,None,"LINESTRING (463181.000 1212313.000, 463361.630..."
1,idDF59D6BB-3E5C-4BFA-91E9-81FAFB70D79C,idE7211001-42DF-416C-9304-D39F9F66D4A6,id8F2F6CFE-2E3D-4CA3-AD41-124E098F7197,None,None,False,Unknown,Minor Road,Single Carriageway,525,...,False,False,False,None,None,None,None,None,None,"LINESTRING (464759.470 1212349.740, 464729.000..."
2,id92D440D3-B707-45F8-8173-E7EE22663573,idCB2E6127-8894-4372-8B1A-D8EECC2B6F8C,id9404ED22-2647-483C-81B6-C294CF3B8712,None,None,False,Unknown,Minor Road,Single Carriageway,338,...,False,False,False,None,None,None,None,None,None,"LINESTRING (463644.000 1212504.000, 463805.030..."
3,id7AA3E18E-21A4-4FD4-B308-EA0B3EA67046,id1CC69224-6F4C-4706-A658-2175240DE21A,id63BA7519-6915-4553-BEA5-AF9C57EF73A5,None,None,False,Unknown,Restricted Local Access Road,Single Carriageway,99,...,False,False,False,None,None,None,None,None,None,"LINESTRING (464193.000 1212455.000, 464217.080..."
4,idB9590DD5-48C1-47F6-9B1A-044C454CCB00,id63BA7519-6915-4553-BEA5-AF9C57EF73A5,idE7211001-42DF-416C-9304-D39F9F66D4A6,None,osgb4000000003210673,False,Unknown,Minor Road,Single Carriageway,104,...,False,False,False,None,Beach Road,None,None,None,None,"LINESTRING (464287.780 1212496.830, 464240.390..."


In [56]:
set(gdf.formOfWay)

{'Collapsed Dual Carriageway',
 'Dual Carriageway',
 'Guided Busway',
 'Roundabout',
 'Shared Use Carriageway',
 'Single Carriageway',
 'Slip Road'}

In [58]:
gdf.to_parquet('openroads.pq')